In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions
from IPython.display import display, Math, Latex
import numpy_financial as npf
import matplotlib.pyplot as plt
import random
from datetime import datetime

To change: 
- convert exchange rates at the beginning
- take monthly values instead
- flip x and y column headers
- implement transaction cost considerations

In [186]:
# cleaning function

tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name, header = None, names=['Ticker'])

total_investment = 1000000
start_date = "2023-10-01"
end_date = "2024-11-20"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        # check if ticker is available
        if ticker.info.get('quoteType')=='NONE':
            # print("phewww", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        # check if its in US or CAD market
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            # print("hmmm", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18] # how does this work?
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            # print("sheesh", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]

    except:
            # print("ufff", tick)
            pass
        
tickers_file_original = tickers_file_original.reset_index(drop=True)

In [187]:
print(tickers_file_original.Ticker.tolist())

['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [190]:
exchange = 'USDCAD=X'
exchange_ticker = yf.Ticker(exchange)
exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
exchange_rate = pd.DataFrame(exchange_rate)
display(exchange_rate)

,Close
Date,
2023-10-01 00:00:00+01:00,1.38286
2023-11-01 00:00:00+00:00,1.35902
2023-12-01 00:00:00+00:00,1.32610
2024-01-01 00:00:00+00:00,1.34017
2024-02-01 00:00:00+00:00,1.35775
2024-03-01 00:00:00+00:00,1.35390
2024-04-01 00:00:00+01:00,1.36667
2024-05-01 00:00:00+01:00,1.36833
2024-06-01 00:00:00+01:00,1.36920


In [192]:
# Define stock list and market index
stock_list = tickers_file_original.Ticker.tolist()
market_indices = ["^GSPC", '^GSPTSE']  # S&P 500

# Step 1: Fetch historical data
data = yf.download(stock_list + market_indices, interval = '1mo', start=start_date, end=end_date)["Close"]
data.index = data.index.strftime('%Y-%m-%d')

# Separate stock prices and market index prices
stock_prices = data[stock_list]
SP = data[market_indices[0]]
TSX = data[market_indices[1]]
display(data)

[*********************100%***********************]  39 of 39 completed


Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB,^GSPC,^GSPTSE
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-01,170.770004,141.179993,94.550003,297.089996,61.310001,133.089996,146.029999,186.820007,26.340000,4.99,...,65.489998,22.360001,77.459999,142.009995,535.559998,207.610001,141.250000,31.879999,4193.799805,18873.500000
2023-11-01,189.949997,142.389999,104.290001,333.140015,65.809998,146.089996,170.770004,231.630005,30.490000,4.98,...,98.849998,24.280001,82.739998,152.710007,552.969971,225.270004,151.610001,38.119999,4567.799805,20236.300781
2023-12-01,192.529999,154.970001,110.070000,350.910004,67.750000,151.940002,187.339996,260.660004,33.669998,4.70,...,103.160004,23.580000,85.620003,170.460007,526.469971,245.619995,157.229996,43.279999,4769.830078,20958.400391
2024-01-01,184.399994,164.399994,113.150002,363.880005,69.510002,155.199997,200.740005,211.039993,34.009998,3.77,...,107.629997,24.080000,81.669998,160.119995,511.739990,243.929993,141.899994,41.540001,4845.649902,21021.900391
2024-02-01,180.750000,176.050003,118.639999,374.779999,72.889999,176.759995,219.419998,203.720001,34.520000,3.80,...,103.690002,23.670000,81.489998,167.330002,493.600006,253.690002,148.259995,41.959999,5096.270020,21363.599609
2024-03-01,171.479996,182.100006,113.660004,346.609985,78.169998,180.380005,227.690002,192.990005,37.919998,3.71,...,104.500000,21.670000,81.750000,174.210007,494.700012,245.929993,148.630005,44.700001,5254.350098,22167.000000
2024-04-01,170.330002,162.639999,105.970001,300.910004,75.309998,175.000000,234.029999,167.839996,37.009998,3.85,...,96.650002,22.110001,81.669998,176.419998,483.700012,237.160004,147.479996,40.630001,5035.689941,21714.500000
2024-05-01,192.250000,161.240005,102.190002,282.290009,78.820000,176.440002,240.000000,177.610001,39.990002,3.80,...,80.660004,22.410000,76.199997,195.009995,495.369995,232.820007,138.929993,40.549999,5277.509766,22269.099609
2024-06-01,210.619995,171.520004,103.910004,303.410004,74.239998,193.250000,231.550003,182.009995,39.770000,3.42,...,90.410004,20.709999,75.199997,194.529999,509.260010,226.259995,136.850006,39.700001,5460.479980,21875.800781


In [194]:
# convert all to CAD
for ticker in stock_list:
    ticker_name = yf.Ticker(ticker)
    currency = ticker_name.fast_info["currency"]
    if currency == 'USD':
        exchange = 'USDCAD=X'
        exchange_ticker = yf.Ticker(exchange)
        exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
        exchange_rate = pd.DataFrame(exchange_rate)
        exchange_rate.index = exchange_rate.index.strftime('%Y-%m-%d')
        data[ticker] = exchange_rate['Close'] * data[ticker]
    elif currency == 'CAD':
        continue
    else:
        raise ValueError(f"Unsupported currency: {currency}")

display(data)

Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB,^GSPC,^GSPTSE
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-01,236.150999,195.232157,130.749412,410.833856,84.783145,184.044825,201.939036,258.345905,36.424531,4.99,...,65.489998,22.360001,77.459999,196.379933,740.604469,287.095554,195.328967,44.085574,4193.799805,18873.500000
2023-11-01,258.145844,193.510856,141.732196,452.743941,89.437103,198.539226,232.079850,314.789808,41.436519,4.98,...,98.849998,24.280001,82.739998,207.535953,751.497247,306.146440,206.041022,51.805841,4567.799805,20236.300781
2023-12-01,255.314030,205.505717,145.963826,465.341753,89.843274,201.487636,248.431568,345.661229,44.649784,4.70,...,103.160004,23.580000,85.620003,226.047014,698.151824,325.716674,208.502696,57.393606,4769.830078,20958.400391
2024-01-01,247.127345,220.323944,151.640240,487.661076,93.155221,207.994384,269.025738,282.829473,45.579180,3.77,...,107.629997,24.080000,81.669998,214.588018,685.818596,326.907665,190.170118,55.670664,4845.649902,21021.900391
2024-02-01,245.413323,239.031902,161.083466,508.857565,98.966401,239.995893,297.917515,276.600844,46.869533,3.80,...,103.690002,23.670000,81.489998,227.192320,670.185437,344.447616,201.300016,56.971191,5096.270020,21363.599609
2024-03-01,232.166759,246.545190,153.884274,469.275244,105.834357,244.216481,308.269484,261.289160,51.339884,3.71,...,104.500000,21.670000,81.750000,235.862920,669.774325,332.964606,201.230157,60.519329,5254.350098,22167.000000
2024-04-01,232.784906,222.275210,144.826023,411.244678,102.923915,239.167252,319.841781,229.381890,50.580455,3.85,...,96.650002,22.110001,81.669998,241.107921,661.058302,324.119465,201.556488,55.527804,5035.689941,21714.500000
2024-05-01,263.061443,220.629537,139.829646,386.265888,107.851770,241.428149,328.399200,243.029092,54.719519,3.80,...,80.660004,22.410000,76.199997,266.838026,677.829626,318.574601,190.102077,55.485781,5277.509766,22269.099609
2024-06-01,288.380895,234.845188,142.273576,415.428974,101.649404,264.597898,317.038262,249.208083,54.453084,3.42,...,90.410004,20.709999,75.199997,266.350473,697.278801,309.795182,187.375027,54.357241,5460.479980,21875.800781


In [196]:
# Step 2: Calculate beta and covariance
returns = stock_prices.pct_change().dropna()
SP_returns = SP.pct_change().dropna()
TSX_returns = TSX.pct_change().dropna()

# Calculate betas
SP_betas = {}
TSX_betas = {}
betas_combined = {}
for stock in stock_list:
    cov1 = np.cov(returns[stock], SP_returns)[0][1]
    var1 = np.var(SP_returns)
    SP_betas[stock] = np.round(cov1 / var1, 3)

    cov2 = np.cov(returns[stock], TSX_returns)[0][1]
    var2 = np.var(TSX_returns)
    TSX_betas[stock] = np.round(cov2 / var2, 3)

    betas_combined[stock] = np.round(SP_betas[stock] + TSX_betas[stock], 3)

# Select stocks with beta close to 1
beta_target = 1
selected_stocks = sorted(betas_combined.keys(), key=lambda x: abs(betas_combined[x] - beta_target))[:18]
print("Combined beta values: ")
display(betas_combined)
print("Selected stocks: ")
print(selected_stocks)

Combined beta values: 


{'AAPL': 1.642,
 'ABBV': 0.775,
 'ABT': 1.939,
 'ACN': 2.862,
 'AIG': 1.915,
 'AMZN': 1.656,
 'AXP': 2.63,
 'BA': 5.655,
 'BAC': 2.923,
 'BB.TO': 1.133,
 'BIIB': 0.274,
 'BK': 2.062,
 'BLK': 4.561,
 'BMY': 2.033,
 'C': 2.837,
 'CAT': 3.642,
 'CL': 0.596,
 'KO': 0.713,
 'LLY': -0.296,
 'LMT': 0.647,
 'MO': 0.509,
 'MRK': 0.269,
 'PEP': 0.772,
 'PFE': 1.503,
 'PG': 0.024,
 'PM': 0.177,
 'PYPL': 2.096,
 'QCOM': 3.517,
 'RY.TO': 2.518,
 'SHOP.TO': 6.358,
 'T.TO': 1.159,
 'TD.TO': 1.857,
 'TXN': 1.812,
 'UNH': 1.164,
 'UNP': 2.339,
 'UPS': 1.378,
 'USB': 4.341}

Selected stocks: 
['BB.TO', 'T.TO', 'UNH', 'ABBV', 'PEP', 'KO', 'LMT', 'UPS', 'CL', 'MO', 'PFE', 'AAPL', 'AMZN', 'BIIB', 'MRK', 'TXN', 'PM', 'TD.TO']


In [198]:
# Step 3: Prepare for portfolio optimization
selected_prices = stock_prices[selected_stocks] # takes the stock prices of the selected stocks, DataFrame
#display(selected_prices)
mu = expected_returns.mean_historical_return(selected_prices) # calculates mean historical returns
S = risk_models.sample_cov(selected_prices) # calculates covariance of each pairof stocks - how stock returns move together
#display(S)

# Optimize portfolio using the EfficientFrontier object
ef = EfficientFrontier(mu, S) # to optimize portfolio weights
ef.add_objective(objective_functions.L2_reg, gamma=0.1)  # Add L2 regularization 
    # to prevent any single stock from having excessive large weight, gamma controls strenght of penalty 
    # figure out what L2_reg is exactly


#ef.add_transaction_costs(selected_prices.iloc[-1], 0.01)  # 1% transaction cost
    # attempts to include transaction costs
weights = ef.efficient_return(target_return=market_returns.mean()) # find optimal weights to achieve market return
weights

/opt/anaconda3/lib/python3.12/site-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pypfopt/risk_models.py:89: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


OrderedDict([('BB.TO', 0.0641143933345876),
             ('T.TO', 0.0668575393128976),
             ('UNH', 0.0617967854085254),
             ('ABBV', -3e-16),
             ('PEP', 0.045615140052054),
             ('KO', 5e-16),
             ('LMT', 3e-16),
             ('UPS', 0.0076042700305082),
             ('CL', 0.0),
             ('MO', 0.1091692696073305),
             ('PFE', -9e-16),
             ('AAPL', 0.0),
             ('AMZN', 0.1903641994508604),
             ('BIIB', 0.0837549836316506),
             ('MRK', 0.1708951454904347),
             ('TXN', 0.0),
             ('PM', 0.1998282736811522),
             ('TD.TO', 5e-16)])

In [202]:
# Step 4: Allocate funds
total_money = 1000000
final_allocation = {}
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        price = ticker.fast_info["lastPrice"]  # need to make sure "end_date" is the most recent trading day
        '''
        currency = ticker.fast_info["currency"]     
        if currency == 'USD':
            exchange = 'USDCAD=X'
            exchange_ticker = yf.Ticker(exchange)
            exchange_rate = exchange_ticker.fast_info["lastPrice"]
            price *= exchange_rate
        elif currency == 'CAD':
            continue
        '''
        num_shares = (allocation / price)
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price
        #transaction_cost = total_cost * 0.01
        final_allocation[stock] = {
            "Number of Shares": num_shares,
            "Transaction Cost": transaction_cost,
            "Total Money Spent": total_cost + transaction_cost,
            "Cumulative Total": np.round(total_cost + transaction_cost+ sum(stock["Total Money Spent"] for stock in final_allocation.values()), 2)
        }
display(pd.DataFrame(final_allocation))
'''
# Print final allocation
print("Final Portfolio Allocation:")
for stock, allocation in final_allocation.items():
    print(f"{stock}: {np.round(allocation['num_shares'], 2)} shares, ${allocation['allocated_money']:.2f}")
'''

print(f"Total Money Spent: ${sum(a['Total Money Spent'] for a in final_allocation.values()):.2f}")


,BB.TO,T.TO,UNH,PEP,KO,LMT,UPS,MO,AMZN,BIIB,MRK,PM,TD.TO
Number of Shares,19787.173810,3085.115550,102.908714,287.355746,7.937642e-12,5.610298e-13,57.568421,1950.112904,938.304695,536.887479,1753.831972,1532.531192,6.391328e-12
Transaction Cost,3.950000,3.085116,0.102909,0.287356,7.937642e-15,5.610298e-16,0.057568,1.950113,0.938305,0.536887,1.753832,1.532531,6.391328e-15
Total Money Spent,64114.393335,66857.539313,61796.785409,45615.140052,5.000000e-10,3.000000e-10,7604.270031,109169.269607,190364.199451,83754.983632,170895.145490,199828.273681,5.000000e-10
Cumulative Total,64114.390000,130971.930000,192768.720000,238383.860000,2.383839e+05,2.383839e+05,245988.130000,355157.400000,545521.600000,629276.580000,800171.730000,1000000.000000,1.000000e+06


Total Money Spent: $1000000.00
